# This notebook focuses on fine-tuning the Vidgen et al. (2021) model using cross-validation and averaging weights.

Importing required libraries and modules.

In [ ]:
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch

Loading data.

In [ ]:
data = pd.read_csv('Notebook_8_9_10_fine_tune_final.csv')

In [ ]:
!pip install transformers

Importing required libraries and modules and Vidgen et al's (2021) model.

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

# 1. Load the pre-trained model and tokenizer
model_name = "facebook/roberta-hate-speech-dynabench-r4-target"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Logging into Huggingface to allow for fine-tune model to be uploaded onto account.

In [ ]:
!pip install huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /roo

In [ ]:
!huggingface-cli whoami

EZiisk


In [ ]:
!pip install accelerate -U transformers[torch]

Importing required libraries and modules and preprocessing sentences.

In [ ]:
import re
import string

def preprocess_sentence(sentence):
  # no lowercasing or punctuation removal as assumed to carry semantic information
    sentence = re.sub(r'\\n', ' ', sentence)
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    return sentence

# Apply the preprocess_sentence function to the 'sentences' column
data['sentences'] = data['sentences'].apply(preprocess_sentence)


Creating the train/validation/test split by stratifying the data using the gold label column.

In [ ]:
# Initialize lists to store train, validate, and test data
train_data, val_data, test_data = [], [], []

# Group data by "gold_label" and create lists of texts and labels for each group
grouped_data = data.groupby('gold_label')
grouped_texts = [group['sentences'].tolist() for _, group in grouped_data]
grouped_labels = [group['hate_label'].tolist() for _, group in grouped_data]

# Split each group into train (70%), validate (15%), and test (15%) sets
for texts, labels in zip(grouped_texts, grouped_labels):
    # Split into train (85%) and test (15%)
    train_texts_group, test_texts_group, train_labels_group, test_labels_group = train_test_split(
        texts, labels, test_size=0.15, stratify=labels, random_state=42
    )

    # Split train set into train (82.35%) and validate (17.65%) to achieve a 70-15-15 split
    train_texts_group, val_texts_group, train_labels_group, val_labels_group = train_test_split(
        train_texts_group, train_labels_group, test_size=0.1765, stratify=train_labels_group, random_state=42
    )

    train_data.extend(list(zip(train_texts_group, train_labels_group)))
    val_data.extend(list(zip(val_texts_group, val_labels_group)))
    test_data.extend(list(zip(test_texts_group, test_labels_group)))

# Separate the train, validate, and test texts and labels
train_texts, train_labels = zip(*train_data)
val_texts, val_labels = zip(*val_data)
test_texts, test_labels = zip(*test_data)



Creating the custom DataLoader.

In [ ]:
# Define a custom dataset class for hate speech detection using PyTorch
class HateSpeechDataset(torch.utils.data.Dataset):

    # Initialize the dataset object
    def __init__(self, texts, labels, tokenizer, max_len):
        # Store the list of textual samples
        self.texts = texts
        # Store the list of labels corresponding to each text sample
        self.labels = labels
        # Store the tokenizer instance which will convert text to tokens
        self.tokenizer = tokenizer
        # Store the maximum token length for sequences
        self.max_len = max_len

    # Return the total number of samples in the dataset
    def __len__(self):
        return len(self.texts)

    # Fetch and return a single data sample given its index
    def __getitem__(self, item):
        # Retrieve the text and its corresponding label using the provided index
        text = self.texts[item]
        label = self.labels[item]

        # Tokenize the text using the provided tokenizer
        # This converts the text to a format suitable for model input
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,   # Add special tokens like [CLS], [SEP]
            max_length=self.max_len,   # Ensure the sequence doesn't exceed the max length
            padding='max_length',      # Pad short sequences to the max length
            truncation=True,           # Truncate sequences exceeding the max length
            return_tensors='pt'        # Return data as PyTorch tensors
        )

        # Return a dictionary containing the tokenized data and the label
        return {
            # The token IDs of the text
            'input_ids': encoding['input_ids'].flatten(),
            # A mask to indicate real tokens (1) vs padded tokens (0)
            'attention_mask': encoding['attention_mask'].flatten(),
            # The corresponding label of the text sample
            'labels': torch.tensor(label, dtype=torch.long)
        }


Defining the training arguments.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

from transformers import EarlyStoppingCallback

#Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=500,
    evaluation_strategy='steps',
    eval_steps=500,
    load_best_model_at_end=True,  # Set load_best_model_at_end to True
)

max_len = 128

Defining the function to compute the accuracy and f1 scores.

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from transformers import EvalPrediction
import numpy as np

# Define evaluation metrics function
def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return {
        'accuracy': accuracy_score(p.label_ids, preds),
        'f1': f1_score(p.label_ids, preds, average='weighted')
    }

Importing required libraries and modules, and running the training and evaluation loops using the Huggingface Trainer class with k-fold cross validation.

In [ ]:
from sklearn.model_selection import StratifiedKFold

# Create a list to store the state dictionaries of the models for each fold
model_weights = []

# Combine the train and validate sets (texts and labels)
train_val_texts = train_texts + val_texts
train_val_labels = train_labels + val_labels

# Initialize StratifiedKFold
num_splits = 5
skf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)

# Create the k different training and validation splits
folds = []
for train_index, val_index in skf.split(train_val_texts, train_val_labels):
    train_texts_fold = [train_val_texts[i] for i in train_index]
    val_texts_fold = [train_val_texts[i] for i in val_index]
    train_labels_fold = [train_val_labels[i] for i in train_index]
    val_labels_fold = [train_val_labels[i] for i in val_index]

    folds.append((train_texts_fold, train_labels_fold, val_texts_fold, val_labels_fold))

# Iterate over the k folds
for fold_idx, (train_texts_fold, train_labels_fold, val_texts_fold, val_labels_fold) in enumerate(folds):
    print(f"Training on Fold {fold_idx + 1}")

    # Create the training and validation datasets
    train_dataset_fold = HateSpeechDataset(train_texts_fold, train_labels_fold, tokenizer, max_len)
    val_dataset_fold = HateSpeechDataset(val_texts_fold, val_labels_fold, tokenizer, max_len)

    # Reinitialize the model for each fold
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # Create the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset_fold,
        eval_dataset=val_dataset_fold,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )

    # Train the model on the current fold
    trainer.train()

    # Save the state dictionary of the model for the current fold
    model_weights.append(model.state_dict())

    # Evaluate on the current fold
    eval_result = trainer.evaluate(val_dataset_fold)
    print(f"Fold {fold_idx + 1} - Validation Accuracy: {eval_result['eval_accuracy']:.4f}, Validation F1: {eval_result['eval_f1']:.4f}")



Training on Fold 1


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1
500,0.460500,0.431319,0.846878,0.846420
1000,0.334700,0.394549,0.853157,0.854008
1500,0.276400,0.378301,0.877921,0.877891
2000,0.197700,0.404660,0.884548,0.884364


Fold 1 - Validation Accuracy: 0.8779, Validation F1: 0.8779
Training on Fold 2


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

Fold 4 - Validation Accuracy: 0.8661, Validation F1: 0.8664
Training on Fold 5


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1
500,0.452400,0.400665,0.833915,0.835107
1000,0.318400,0.474451,0.856246,0.854193
1500,0.260700,0.632115,0.854501,0.855506


Fold 5 - Validation Accuracy: 0.8339, Validation F1: 0.8351


NameError: ignored

Processing data.

In [ ]:
# Step 6: Evaluate on Test Set

test_dataset = HateSpeechDataset(test_texts, test_labels, tokenizer, max_len)

eval_result = trainer.evaluate(test_dataset)
print(f"Test Accuracy: {eval_result['eval_accuracy']:.4f}, Test F1: {eval_result['eval_f1']:.4f}")

# Get the predicted labels for the test dataset
test_predictions = trainer.predict(test_dataset).predictions
test_predicted_labels = np.argmax(test_predictions, axis=1)

# Create a dictionary to store the collected information
test_data_dict = {
    "original_sentence": test_texts,  # Original sentences from the test dataset
    "hate_label": test_labels,  # Ground truth hate labels from the HateSpeechDataset
    "predicted_label": test_predicted_labels,  # Predicted labels from the model
}

# Create a DataFrame from the dictionary
test_results_df = pd.DataFrame(test_data_dict)

Test Accuracy: 0.8346, Test F1: 0.8358


Averaging the weights from the cross validation process.

In [ ]:
from collections import OrderedDict

def average_model_weights(model_weights):
    # Initialize a dictionary to store the sum of the weights
    avg_weights = OrderedDict()

    # Iterate through each state dictionary (weights for each fold)
    for state_dict in model_weights:
        for key, value in state_dict.items():
            # If the key is not in avg_weights, initialize it with zeros
            if key not in avg_weights:
                avg_weights[key] = torch.zeros_like(value)

            # Add the value to the corresponding key in avg_weights
            avg_weights[key] += value

    # Divide by the number of models to obtain the average
    for key in avg_weights:
        avg_weights[key] /= len(model_weights)

    return avg_weights

# Get the average weights
avg_weights = average_model_weights(model_weights)

# Create a new model instance and load the averaged weights
ensemble_model = AutoModelForSequenceClassification.from_pretrained(model_name)
ensemble_model.load_state_dict(avg_weights)


<All keys matched successfully>

Saving the averaged weights model to the HuggingFace Hub.

In [ ]:
# Saving new ensemble model with averaged weights to HuggingFace Hub

ensemble_model.save_pretrained("EZiisk/EZ_finetune_Vidgen_model_RHS_ensemble", push_to_hub = True)
tokenizer.save_pretrained("EZiisk/EZ_finetune_Vidgen_model_RHS_ensemble_tokenizer", push_to_hub = True)

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

('EZiisk/EZ_finetune_Vidgen_model_RHS_ensemble_tokenizer/tokenizer_config.json',
 'EZiisk/EZ_finetune_Vidgen_model_RHS_ensemble_tokenizer/special_tokens_map.json',
 'EZiisk/EZ_finetune_Vidgen_model_RHS_ensemble_tokenizer/vocab.json',
 'EZiisk/EZ_finetune_Vidgen_model_RHS_ensemble_tokenizer/merges.txt',
 'EZiisk/EZ_finetune_Vidgen_model_RHS_ensemble_tokenizer/added_tokens.json',
 'EZiisk/EZ_finetune_Vidgen_model_RHS_ensemble_tokenizer/tokenizer.json')

Producing the dataframe of the test sentences, their ground truth labels and the predicted labels from the model.

In [ ]:
# Inner merge based on the condition where 'sentences' matches 'original_sentence'
merged_df = data.merge(test_results_df, left_on='sentences', right_on='original_sentence', how='inner')

# Drop the duplicate 'original_sentence' column after the merge
merged_df.drop('original_sentence', axis=1, inplace=True)

In [ ]:
columns_to_drop = ['hate_label_y']
merged_df = merged_df.drop(columns_to_drop, axis=1)

In [ ]:
column_name_mapping = {
    'hate_label_x': 'hate_label',
}

# Rename the columns using the dictionary
merged_df.rename(columns=column_name_mapping, inplace=True)

Processing data.

In [ ]:
columns_to_drop = ['clean_sentences']
merged_df = merged_df.drop(columns_to_drop, axis=1)

Importing required libraries and modules.

In [ ]:
from google.colab import files

merged_df.to_csv("2.59_finetune_test_dataset_analysis", index=False)
files.download("2.59_finetune_test_dataset_analysis")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>